In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

from bnn_layers import BinarizeLinear

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.__version__)

C:\Users\lubai\anaconda3\envs\pytorch\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


2.2.0


In [2]:
# hyper parameters

# 训练参数
batch_size = 256
epochs = 10
# learning rate
lr_start = 1e-3
loss_fn = nn.CrossEntropyLoss()

# 导入 MNIST 数据集

In [3]:
# 图像预处理
# 将图像从 [0.0, 1.0] 范围内的像素值转换到 [-1.0, 1.0]
transform = transforms.ToTensor()

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [4]:
# # 将dataset放入DataLoader数据加载器
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

for X, y in test_dataloader:
    # 分别代表批次大小（Number of samples in the batch）、通道数（Channels）、高度（Height）和宽度（Width）
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([256, 1, 28, 28])
Shape of y: torch.Size([256]) torch.int64


# 模型定义

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            BinarizeLinear(28 * 28, 20, is_binarized_activation=False),
            BinarizeLinear(20, 10),
        )

    def forward(self, x):
        output = self.layers(x)
        return output

In [6]:
# 构建模型。并移至CUDA
model = Net().to(device)
print(model)

Net(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): BinarizeLinear(in_features=784, out_features=20, bias=False)
    (2): BinarizeLinear(in_features=20, out_features=10, bias=False)
  )
)


# 模型训练

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    # 设置模型为训练模式，以启用参数更新
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 将数据移至CUDA
        X, y = X.to(device), y.to(device)

        # 前向传播
        ## 计算模型output输出
        pred = model(X)
        ## 计算损失值
        loss = loss_fn(pred, y)

        # 反向传播
        ## 计算梯度
        loss.backward()
        ## 使用optimizer，，基于之前计算得到的梯度，更新模型权重Weights和偏置Biases
        optimizer.step()
        ## 清零梯度。避免梯度累积的问题。为下一次迭代准备
        optimizer.zero_grad()

        # 每100个批次打印一次损失信息
        if batch % 100 == 0:
            size = len(dataloader.dataset)  # 数据集总大小
            loss = loss.item()
            current = (batch + 1) * len(X)  # 目前为止处理过的数据总量。len(X) 是当前批次数据的数量
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    # 数据集总大小
    size = len(dataloader.dataset)
    # 批次总数
    num_batches = len(dataloader)
    # 将模型设置成评估模式。这样不会更新参数。
    model.eval()
    test_loss, correct = 0, 0
    # 在评估时，禁用梯度计算。减少计算和内存消耗
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # 累加批次的损失值
            test_loss += loss_fn(pred, y).item()
            # 累加正确预测的数量
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches  # 计算平均损失
    correct /= size  # 计算准确率
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_start)

In [10]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 6.439171  [  256/60000]
loss: 0.949076  [25856/60000]
loss: 0.997286  [51456/60000]
Test Error: 
 Accuracy: 76.3%, Avg loss: 0.769493 

Epoch 2
-------------------------------
loss: 0.936751  [  256/60000]
loss: 0.917794  [25856/60000]
loss: 0.631150  [51456/60000]
Test Error: 
 Accuracy: 79.5%, Avg loss: 0.711927 

Epoch 3
-------------------------------
loss: 0.645477  [  256/60000]
loss: 0.692262  [25856/60000]
loss: 0.806472  [51456/60000]
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.758508 

Epoch 4
-------------------------------
loss: 0.950614  [  256/60000]
loss: 0.887840  [25856/60000]
loss: 0.564849  [51456/60000]
Test Error: 
 Accuracy: 76.8%, Avg loss: 0.781680 

Epoch 5
-------------------------------
loss: 0.866059  [  256/60000]
loss: 0.736498  [25856/60000]
loss: 0.836463  [51456/60000]
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.636810 

Epoch 6
-------------------------------
loss: 0.572466  [  256/60000]
loss: 0.718001  [258